In [1]:
import requests
import shutil
import pandas as pd
import geopandas as gpd
import os
from shapely.geometry import Point

In [2]:
NDAWN_Geofolder_new = r'C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3'
if not os.path.exists(NDAWN_Geofolder_new):
    os.mkdir(NDAWN_Geofolder_new)
    
#List for URL to iterate
variable = ['ddavt', 'ddmxt', 'ddmnt']

for Var in variable:
    #retrieves CSV data from the NDAWN website for weather stations and date range
    dataurl = fr'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=159&station=10&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=201&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=192&station=19&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=172&station=139&station=158&station=23&station=157&station=62&station=86&station=24&station=89&station=126&station=167&station=93&station=183&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=153&station=105&station=82&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=193&station=71&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=69&station=194&station=145&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=57&station=149&station=148&station=202&station=110&variable={Var}&ttype=daily&quick_pick=30_d'
    NDAWNoutput = fr'C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\NDAWN_{Var}.csv'
    NDAWN_Geo_Get = requests.get(dataurl, verify=False, stream=True) #get request funciton with url
    #saving the downloaded data to the local file system.
    with open(NDAWNoutput, 'wb') as file:
        file.write(NDAWN_Geo_Get.content)
   #Clean Data
    NDAWNoutput = fr'C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\NDAWN_{Var}.csv'
    
    #The provided code reads a CSV file into a pandas DataFrame, skips the first 4 rows in the CSV file,and displays the DataFrame with new column names.
    North_Dogg_Dataframe = pd.read_csv(fr'C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\NDAWN_{Var}.csv', skiprows=4)
    new_names = ['Location', 'Latitude', 'Longitude', 'Elevation', 'Year', 'Month', 'Day', f'{Var}', 'Flag']
    North_Dogg_Dataframe.columns = new_names
    North_Dogg_Dataframe

    # Group the data by the 'Location' column and calculate the average temperature for each location
    monthly_avg_temp = North_Dogg_Dataframe.groupby(['Latitude', 'Longitude', 'Location'])[f'{Var}'].mean()

    # Convert to DataFrame
    monthly_avg_temp_df = monthly_avg_temp.reset_index()

    # Save the result to a CSV file
    output_csv = fr'C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\monthly_avg_temperatures{Var}.csv'
    monthly_avg_temp_df.to_csv(output_csv, index=False)

    # Print the result
    print(monthly_avg_temp_df)

    #creates pionts on map hehe
    arcpy.management.XYTableToPoint(
        in_table=fr"C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\monthly_avg_temperatures{Var}.csv",
        out_feature_class=fr"C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\monthly_avg_temperatures{Var}_XYTableToPoint",
        x_field="Longitude",
        y_field="Latitude",
        z_field=None,
        coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
    )    
    

C:\Users\Track\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ndawn.ndsu.nodak.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


     Latitude  Longitude      Location      ddavt
0    44.68768  -92.85985      Hastings  44.589867
1    45.25334  -95.97341      Holloway  42.329367
2    45.34399  -93.85014        Becker  43.386867
3    45.71509  -95.17179      Westport  40.773833
4    45.79384  -94.26182          Rice  42.007567
..        ...        ...           ...        ...
188  48.88351  -97.15029      Humboldt  36.049833
189  48.89141  -98.62842         Wales  33.915000
190  48.96375 -103.80585       Fortuna  36.064900
191  48.98128 -102.52419        Portal  34.843933
192  48.99821 -100.06389  Peace Garden  33.733700

[193 rows x 4 columns]


C:\Users\Track\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ndawn.ndsu.nodak.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


     Latitude  Longitude      Location      ddmxt
0    44.68768  -92.85985      Hastings  53.571967
1    45.25334  -95.97341      Holloway  53.248867
2    45.34399  -93.85014        Becker  52.395800
3    45.71509  -95.17179      Westport  50.177100
4    45.79384  -94.26182          Rice  51.065500
..        ...        ...           ...        ...
188  48.88351  -97.15029      Humboldt  44.040200
189  48.89141  -98.62842         Wales  42.234733
190  48.96375 -103.80585       Fortuna  44.430567
191  48.98128 -102.52419        Portal  43.669833
192  48.99821 -100.06389  Peace Garden  40.583233

[193 rows x 4 columns]


C:\Users\Track\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ndawn.ndsu.nodak.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


     Latitude  Longitude      Location      ddmnt
0    44.68768  -92.85985      Hastings  35.607333
1    45.25334  -95.97341      Holloway  31.409433
2    45.34399  -93.85014        Becker  34.377600
3    45.71509  -95.17179      Westport  31.370133
4    45.79384  -94.26182          Rice  32.949200
..        ...        ...           ...        ...
188  48.88351  -97.15029      Humboldt  28.059000
189  48.89141  -98.62842         Wales  25.594833
190  48.96375 -103.80585       Fortuna  27.698833
191  48.98128 -102.52419        Portal  26.017533
192  48.99821 -100.06389  Peace Garden  26.883767

[193 rows x 4 columns]


In [9]:
#Kriging Minimum
arcpy.ddd.Kriging(
    in_point_features="monthly_avg_temperaturesddmnt_XYTableToPoint",
    z_field="ddmnt",
    out_surface_raster=r"C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\KrigingMin",
    semiVariogram_props="Spherical # # # #",
    cell_size=0.0172421199999999,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)
#Kriging Max
arcpy.ddd.Kriging(
    in_point_features="monthly_avg_temperaturesddmxt_XYTableToPoint",
    z_field="ddmxt",
    out_surface_raster=r"C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\KrigingMAX",
    semiVariogram_props="Spherical # # # #",
    cell_size=0.0172421199999999,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)
#Kriging Avg
arcpy.ddd.Kriging(
    in_point_features="monthly_avg_temperaturesddavt_XYTableToPoint",
    z_field="ddavt",
    out_surface_raster=r"C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\KrigingAVG",
    semiVariogram_props="Spherical # # # #",
    cell_size=0.0172421199999999,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)

<Result 'C:\\Users\\Track\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab3\\Lab3.gdb\\KrigingAVG'>

In [10]:
#Diffusion Min
arcpy.ga.DiffusionInterpolationWithBarriers(
    in_features="monthly_avg_temperaturesddmnt_XYTableToPoint",
    z_field="ddmnt",
    out_ga_layer=None,
    out_raster=r"C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\DiffusionMin",
    cell_size=0.0172421199999999,
    in_barrier_features=None,
    bandwidth=None,
    number_iterations=100,
    weight_field=None,
    in_additive_barrier_raster=None,
    in_cumulative_barrier_raster=None,
    in_flow_barrier_raster=None
)
#Diffusion Max
arcpy.ga.DiffusionInterpolationWithBarriers(
    in_features="monthly_avg_temperaturesddmxt_XYTableToPoint",
    z_field="ddmxt",
    out_ga_layer=None,
    out_raster=r"C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\DiffusionMAX",
    cell_size=0.0172421199999999,
    in_barrier_features=None,
    bandwidth=None,
    number_iterations=100,
    weight_field=None,
    in_additive_barrier_raster=None,
    in_cumulative_barrier_raster=None,
    in_flow_barrier_raster=None
)
#Diffusion Average
arcpy.ga.DiffusionInterpolationWithBarriers(
    in_features="monthly_avg_temperaturesddavt_XYTableToPoint",
    z_field="ddavt",
    out_ga_layer=None,
    out_raster=r"C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\DiffusionAVG",
    cell_size=0.0172421199999999,
    in_barrier_features=None,
    bandwidth=None,
    number_iterations=100,
    weight_field=None,
    in_additive_barrier_raster=None,
    in_cumulative_barrier_raster=None,
    in_flow_barrier_raster=None
)

<Result ''>

In [11]:
#IDW MIN
arcpy.ddd.Idw(
    in_point_features="monthly_avg_temperaturesddmnt_XYTableToPoint",
    z_field="ddmnt",
    out_raster=r"C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\IdwMin",
    cell_size=0.0172421199999999,
    power=2,
    search_radius="VARIABLE 12",
    in_barrier_polyline_features=None
)
#IDW Max
arcpy.ddd.Idw(
    in_point_features="monthly_avg_temperaturesddmxt_XYTableToPoint",
    z_field="ddmxt",
    out_raster=r"C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\IdwMAX",
    cell_size=0.0172421199999999,
    power=2,
    search_radius="VARIABLE 12",
    in_barrier_polyline_features=None
)
#IDW Avg
arcpy.ddd.Idw(
    in_point_features="monthly_avg_temperaturesddavt_XYTableToPoint",
    z_field="ddavt",
    out_raster=r"C:\Users\Track\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\IdwAVG",
    cell_size=0.0172421199999999,
    power=2,
    search_radius="VARIABLE 12",
    in_barrier_polyline_features=None
)

<Result 'C:\\Users\\Track\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab3\\Lab3.gdb\\IdwAVG'>